trec-covid 데이터 중 10,000개만 generate하고 10,000개 필터링
- generator는 gpt2를 우선 사용 (논문에서 사용한 모델의 크기가 너무 커서 돌아가지않음)
- filtering-reranker는 monot5-msmarco 사용

In [1]:
import json
import pandas as pd

In [2]:
# filtered by reranker
with open('/content/InPars/trec-covid-queries-filtered-reranker.jsonl', 'r') as f:
    reranker = [json.loads(line.strip()) for line in f]

with open('/content/InPars/trec-covid-queries-filtered.jsonl', 'r') as f:
    scores = [json.loads(line.strip()) for line in f]

In [3]:
df_reranker = pd.DataFrame(reranker)
df_scores = pd.DataFrame(scores)

In [4]:
df_reranker.head()

,doc_id,text,query,log_probs,prompt_text,fewshot_examples,score
0,ppkjimsr,Lack of Association between Toxocara Exposure ...,Is Toxocara exposure associated with suicide a...,"[-1.8166085482, -1.291793704, -0.0210245755, -...",Example 1:\nDocument: We don't know a lot abou...,"[42, 8, 22]",0.999451
1,4nbh8dcu,ECLS in pertussis: does it have a role? Object...,Is there a role for ECLS in paediatric patient...,"[-1.736659646, -1.5612772703, -0.8064665794, -...",Example 1:\nDocument: We don't know a lot abou...,"[42, 8, 22]",0.999316
2,pa64tvfh,How did the introduction of managed care for t...,What is the impact of the Iowa Managed Substan...,"[-1.4961897135, -1.3378881216, -0.4612050354, ...",Example 1:\nDocument: We don't know a lot abou...,"[42, 8, 22]",0.999312
3,eu4vlfui,Selection of Suitable Reference Genes for Norm...,What is the most suitable gene set for normali...,"[-1.745452404, -0.9453600049, -0.5376476645, -...",Example 1:\nDocument: We don't know a lot abou...,"[42, 8, 22]",0.999308
4,7nmq9cfi,Reflexive Modernization at the Source: Local M...,What is the role of local media in response to...,"[-1.5502038002, -1.0576927662, -0.6937198043, ...",Example 1:\nDocument: We don't know a lot abou...,"[42, 8, 22]",0.999303


In [5]:
df_scores.head()

,doc_id,text,query,log_probs,prompt_text,fewshot_examples,score
0,tpy75jqj,Isolation and identification of avian rotaviru...,Isolation and identification of avian rotaviru...,"[-1.3865540028, -0.7843101621, -0.555095911, -...",Example 1:\nDocument: We don't know a lot abou...,"[42, 8, 22]",-0.095661
1,lgxavbbt,"Isolation of influenza A virus, subtype H5N2, ...","Isolation of influenza A virus, subtype H5N2, ...","[-1.5761212111, -0.9357976913, -0.0732288361, ...",Example 1:\nDocument: We don't know a lot abou...,"[42, 8, 22]",-0.096500
2,cr6omr9l,How do we … integrate pathogen reduced platele...,How do we … integrate pathogen reduced platele...,"[-1.340043664, -1.5322970152, -0.1015384123, -...",Example 1:\nDocument: We don't know a lot abou...,"[42, 8, 22]",-0.106305
3,xtehuzt8,What is the diagnostic accuracy of single nerv...,What is the diagnostic accuracy of single nerv...,"[-1.5485185385, -0.49334535, -0.1963064373, -1...",Example 1:\nDocument: We don't know a lot abou...,"[42, 8, 22]",-0.106941
4,62k09e1o,¿Qué papel desempeña la atención primaria en e...,¿Qué papel desempeña la atención primaria en e...,"[-1.7085156441, -0.0391803272, -1.2398297787, ...",Example 1:\nDocument: We don't know a lot abou...,"[42, 8, 22]",-0.112181


In [6]:
df_scores[df_scores["doc_id"] == "ppkjimsr"]

,doc_id,text,query,log_probs,prompt_text,fewshot_examples,score
998,ppkjimsr,Lack of Association between Toxocara Exposure ...,Is Toxocara exposure associated with suicide a...,"[-1.8166085482, -1.291793704, -0.0210245755, -...",Example 1:\nDocument: We don't know a lot abou...,"[42, 8, 22]",-0.658807


In [10]:
df_scores[df_scores["doc_id"] == "ppkjimsr"]["text"].values[0]

'Lack of Association between Toxocara Exposure and Suicide Attempts in Psychiatric Patients Infection with Toxocara may affect the central nervous system. A high seroprevalence of Toxocara infection has been reported in psychiatric patients. To the best of our knowledge, there is no previous report about an association of Toxocara infection with suicide attempts. Therefore, we sought to determine whether Toxocara exposure is associated with suicide attempts in psychiatric patients. We studied 282 psychiatric outpatients (156 with suicide attempts and 126 without suicide attempts). Sera of patients were analyzed for the presence of anti-Toxocara IgG antibodies by using a commercially available enzyme immunoassay. One of the 156 (0.6%) suicide attempters and 1 (0.8%) of the 126 controls were positive for anti-Toxocara IgG antibodies (OR = 0.80; 95% CI: 0.04–13.02; P = 1.00). Toxocara seropositivity was significantly higher (P = 0.01) in male patients with consumption of raw dried goat me

In [12]:
df_scores[df_scores["doc_id"] == "ppkjimsr"]["query"].values

array(['Is Toxocara exposure associated with suicide attempts in psychiatric patients?'],
      dtype=object)

In [7]:
# count the number of queries that both reranker and scores have
intersection = pd.merge(df_reranker, df_scores, how='inner', on=['doc_id'])

In [8]:
len(intersection)

227